In [29]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 4.6 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelea

In [30]:
#!/usr/bin/env python

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime as dt
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options =chrome_options)
SOURCE = "https://dashboard.madiunkota.go.id/"

In [31]:
wd.get(SOURCE)
wd.find_element_by_xpath("/html/body/div[3]/div/div[2]/div[2]/div[1]/div[9]/div/div[2]/div/div/div[1]/div[2]/label/select/option[4]").click()
content = wd.page_source
soup = BeautifulSoup(content, features="html.parser")
table = soup.find("table", "table table-bordered table-hover datatable-basic dataTable no-footer")
td = (table.find_all('td'))


In [32]:
def ChangeDatetoRightFormat(string_date):
  kamus = ['januari', 'februari', 'maret', 'april', 'mei', 'juni','juli','agustus', 'september', 'oktober','november','desember']
  date_hari, date_bulan, date_tahun = string_date[:2],string_date[3:-5],string_date[-4:]
  date_bulan = kamus.index(date_bulan.lower())+1
  if(int(date_hari)<10):
    date_hari= "0"+date_hari
  if(date_bulan<10):
    date_bulan="0{}".format(date_bulan)
  return "{}-{}-{}".format(date_tahun,date_bulan,date_hari)

In [52]:
def tabulasi_data(ldict,td):
  td = [x.get_text().strip().capitalize() if x.get_text() !=" " else "0" for x in td]
  ldict["kelurahan"].append(td[1])
  ldict["kecamatan"].append(kecamatan(td[1]))
  ldict["total_odp"].append(int(td[3])) 
  ldict["total_pdp"].append(int(td[4])+ int(td[5]))
  ldict["total_positif"].append(int(td[6]))
  ldict["positif_sembuh"].append(int(td[7]))
  ldict["positif_dirawat"].append(int(td[8]))
  ldict["positif_isolasi"].append(int(td[9]))
  ldict["positif_meninggal"].append(int(td[10]))

In [48]:
def kecamatan(desa):
  desa = desa.lower()
  kartoharjo = ['kartoharjo','kanigoro','kelun','rejomulyo','tawangrejo','klegen','oro-oro ombo','pilangbango','sukosari']
  taman = ['taman','banjarejo','demangan','kejuron','josenan','pandean','manisrejo','mojorejo','kuncen']
  manguharjo = ['manguharjo','pangongangan','madiun lor','patihan','winongo','ngegong','nambangan kidul','sogaten','nambangan lor']
  if desa in kartoharjo:
    return "Kartoharjo"
  elif desa in taman:
    return "Taman"
  elif desa in manguharjo:
    return "Manguharjo"
  else:
    return "Null"

In [35]:
import re
date = soup.find("div",class_="col-sm-12 col-md-12")
date = date.find_all("span")
date[1].get_text()
string_date = re.search(r":(.[\w ]*),",date[1].get_text()).group(1).strip()
formatted_date = ChangeDatetoRightFormat(string_date)
scrape_date = dt.now().strftime('%Y-%m-%d')

In [57]:
keys = ["kecamatan","kelurahan","total_positif","positif_sembuh","positif_dirawat","positif_isolasi","positif_meninggal","total_odp", "total_pdp"]
Ldict = dict(zip(keys, ([] for _ in keys)))

In [58]:
for i in range(0,len(td),12):
  tabulasi_data(Ldict,td[i:i+11])

In [59]:
df = pd.DataFrame()
lenghtDF = len(Ldict["kelurahan"])
df['scrape_date'] = [scrape_date]*lenghtDF
df['date_update'] = [formatted_date]*lenghtDF
df['provinsi'] = ["Jawa Timur"]*lenghtDF
df['kabkot'] = ["Madiun"]*lenghtDF
df['Kecamatan'] = Ldict["kecamatan"]
df["Kelurahan"] = Ldict["kelurahan"]
df['total_odp'] = Ldict["total_odp"]
df['total_pdp'] = Ldict["total_pdp"]
df['total_positif'] = Ldict["total_positif"]
df['positif_sembuh'] = Ldict["positif_sembuh"]
df['positif_dirawat'] = Ldict["positif_dirawat"]
df['positif_isolasi'] = Ldict["positif_isolasi"]
df['positif_meninggal'] = Ldict["positif_meninggal"]
df['source_link'] = [SOURCE]*lenghtDF
df['user_pic'] = ["Zaidan"]*lenghtDF
df

,scrape_date,date_update,provinsi,kabkot,Kecamatan,Kelurahan,total_odp,total_pdp,total_positif,positif_sembuh,positif_dirawat,positif_isolasi,positif_meninggal,source_link,user_pic
0,2021-07-29,2021-07-28,Jawa Timur,Madiun,Taman,Banjarejo,240,0,227,177,27,12,11,https://dashboard.madiunkota.go.id/,Zaidan
1,2021-07-29,2021-07-28,Jawa Timur,Madiun,Taman,Demangan,284,2,234,195,23,6,10,https://dashboard.madiunkota.go.id/,Zaidan
2,2021-07-29,2021-07-28,Jawa Timur,Madiun,Taman,Josenan,176,0,175,137,15,14,9,https://dashboard.madiunkota.go.id/,Zaidan
3,2021-07-29,2021-07-28,Jawa Timur,Madiun,Kartoharjo,Kanigoro,231,2,303,201,49,34,19,https://dashboard.madiunkota.go.id/,Zaidan
4,2021-07-29,2021-07-28,Jawa Timur,Madiun,Kartoharjo,Kartoharjo,79,1,111,80,16,8,7,https://dashboard.madiunkota.go.id/,Zaidan
5,2021-07-29,2021-07-28,Jawa Timur,Madiun,Taman,Kejuron,229,1,237,179,26,13,19,https://dashboard.madiunkota.go.id/,Zaidan
6,2021-07-29,2021-07-28,Jawa Timur,Madiun,Kartoharjo,Kelun,122,0,143,103,15,17,8,https://dashboard.madiunkota.go.id/,Zaidan
7,2021-07-29,2021-07-28,Jawa Timur,Madiun,Kartoharjo,Klegen,282,0,285,230,16,16,23,https://dashboard.madiunkota.go.id/,Zaidan
8,2021-07-29,2021-07-28,Jawa Timur,Madiun,Taman,Kuncen,9,0,16,12,1,1,2,https://dashboard.madiunkota.go.id/,Zaidan
9,2021-07-29,2021-07-28,Jawa Timur,Madiun,Manguharjo,Madiun lor,200,0,235,172,20,24,19,https://dashboard.madiunkota.go.id/,Zaidan
